## Data encoding
This part takes a pre-processed dataframe, and encodes the data so it can be used to train Banksformer.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import time

import pickle
from datetime import date
import tensorflow as tf
from prepare_data import preprocess_data_czech

2023-10-26 12:34:48.143581: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [8]:
from random import randint
randint(1,3)

3

In [2]:
ds_suffix = "-czech"
max_seq_len = 80
min_seq_len = 20

In [3]:
raw_data = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
data = preprocess_data_czech(raw_data)
data2 = data[['account_id',  'amount', 'tcode', 'datetime', 'year', 'month', 'dow', 'day', 'dtme','td','age']]
df = data2.copy()

In [4]:
CAT_FIELDS = ['tcode_num']

In [5]:
# def fit_transform(self, df) from DataEncoder Class
# LOG_AMOUNT_SCALE, TD_SCALE, ATTR_SCALE, START_DATE, TCODE_TO_NUM, NUM_TO_TCODE, n_tcodes   are attributes of class.
df["log_amount"] = np.log10(df["amount"]+1)
LOG_AMOUNT_SCALE = df["log_amount"].std()
df["log_amount_sc"] = df["log_amount"] / LOG_AMOUNT_SCALE

TD_SCALE = df["td"].std()
df["td_sc"] = df["td"] / TD_SCALE

ATTR_SCALE = df["age"].std()
df["age_sc"] = df["age"] / ATTR_SCALE

START_DATE = df["datetime"].min()


        

In [6]:
for field in CAT_FIELDS:
    field = field.replace("_num", "")
    cat_to_num = dict([(tc, i) for i, tc in enumerate(df[field].unique())])
    TCODE_TO_NUM = cat_to_num
    NUM_TO_TCODE = dict([(i, tc) for i, tc in enumerate(df[field].unique())])
    df[field + "_num"] = df[field].apply(lambda x: cat_to_num[x])
    n_tcodes = len(cat_to_num)

In [7]:
df

,account_id,amount,tcode,datetime,year,month,dow,day,dtme,td,age,log_amount,log_amount_sc,td_sc,age_sc,tcode_num
0,1,1000.0,CREDIT__CREDIT IN CASH__nan,1995-03-24,1995,3,4,24,7,0.0,29,3.000434,2.823750,0.000000,1.745524,0
1,1,3679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-04-13,1995,4,3,13,17,20.0,29,3.565848,3.355869,3.298201,1.745524,1
2,1,12600.0,CREDIT__CREDIT IN CASH__nan,1995-04-23,1995,4,6,23,7,10.0,29,4.100405,3.858949,1.649100,1.745524,0
3,1,19.2,CREDIT__nan__INTEREST CREDITED,1995-04-30,1995,4,6,30,0,7.0,29,1.305351,1.228484,1.154370,1.745524,2
4,1,3679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-05-13,1995,5,5,13,18,13.0,29,3.565848,3.355869,2.143831,1.745524,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,11382,25600.0,DEBIT__CASH WITHDRAWAL__nan,1998-12-02,1998,12,2,2,29,2.0,46,4.408257,4.148672,0.329820,2.768763,3
1056316,11382,46248.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1998-12-10,1998,12,3,10,21,8.0,46,4.665102,4.390393,1.319280,2.768763,1
1056317,11382,6300.0,DEBIT__CASH WITHDRAWAL__nan,1998-12-25,1998,12,4,25,6,15.0,46,3.799409,3.575677,2.473651,2.768763,3
1056318,11382,311.3,CREDIT__nan__INTEREST CREDITED,1998-12-31,1998,12,3,31,0,6.0,46,2.494572,2.347677,0.989460,2.768763,2


In [7]:

CONT_FIELDS = ['log_amount_sc']
DATE_FIELDS = ['dow', 'month', "day", 'dtme', 'td_sc']  
DATA_KEY_ORDER = CAT_FIELDS + DATE_FIELDS + CONT_FIELDS
date_loss = "scce"

LOSS_TYPES = {"day": date_loss,
              "dtme": date_loss,
           "dow": date_loss,
           "month": date_loss,
           "tcode_num": date_loss,
            "td_sc": "pdf",
            "log_amount_sc": "pdf",
             }
CLOCK_DIMS = {"day": 31,
              "dtme": 31,
           "dow": 7,
           "month": 12,}

# cl - clock encoding (2d)
# oh - One-hot encoding
# raw - no encoding
# cl-i -  clock integer: transforms [1, 2, ..., n] -> [1, 2, ..., n-1, 0]
INP_ENCODINGS = {"day": "cl",
                 "dtme": "cl",
           "dow": "cl",
           "month": "cl",
            "td_sc": "raw",
            "log_amount_sc": "raw",
            "tcode_num": "oh",}

TAR_ENCODINGS = {"day": "cl-i",
                 "dtme": "cl-i",
           "dow": "cl-i",
           "month": "cl-i",
            "td_sc": "raw",
            "log_amount_sc": "raw",
            "tcode_num": "raw",}

FIELD_DIMS_IN  = {}
FIELD_DIMS_TAR = {}
FIELD_DIMS_NET = {}


ENCODING_INP_DIMS_BY_TYPE = {'cl':2, 
                             'oh':None, 
                             'raw':1}

ENCODING_TAR_DIMS_BY_TYPE = {'cl-i': 1, 
                             'raw': 1}

# ENCODING_NET_DIMS_BY_TYPE 

for k in DATA_KEY_ORDER:
    
    FIELD_DIMS_IN[k] = ENCODING_INP_DIMS_BY_TYPE[INP_ENCODINGS[k]]
    FIELD_DIMS_TAR[k] = ENCODING_TAR_DIMS_BY_TYPE[TAR_ENCODINGS[k]]
    
    if TAR_ENCODINGS[k] == "raw":
        FIELD_DIMS_NET[k] = 2
    elif TAR_ENCODINGS[k] == "cl-i":
        FIELD_DIMS_NET[k] = CLOCK_DIMS[k]
    else:
        raise Exception(f"Error getting network dim for field = {k}")
# field = 'tcode'  
for field in CAT_FIELDS:      
    FIELD_DIMS_IN[field] = n_tcodes
    FIELD_DIMS_NET[field] = n_tcodes

FIELD_STARTS_IN = {}
start = 0
for k in DATA_KEY_ORDER:

    FIELD_STARTS_IN[k] = start
    start += FIELD_DIMS_IN[k]



FIELD_STARTS_TAR = {}
start = 0
for k in DATA_KEY_ORDER:

    FIELD_STARTS_TAR[k] = start
    start += FIELD_DIMS_TAR[k]
    
    
FIELD_STARTS_NET = {}
start = 0
for k in DATA_KEY_ORDER:

    FIELD_STARTS_NET[k] = start
    start += FIELD_DIMS_NET[k]


In [8]:
FIELD_DIMS_IN

{'tcode_num': 16,
 'dow': 2,
 'month': 2,
 'day': 2,
 'dtme': 2,
 'td_sc': 1,
 'log_amount_sc': 1}

In [9]:
FIELD_STARTS_IN

{'tcode_num': 0,
 'dow': 16,
 'month': 18,
 'day': 20,
 'dtme': 22,
 'td_sc': 24,
 'log_amount_sc': 25}

In [10]:
FIELD_DIMS_TAR

{'tcode_num': 1,
 'dow': 1,
 'month': 1,
 'day': 1,
 'dtme': 1,
 'td_sc': 1,
 'log_amount_sc': 1}

In [11]:
FIELD_STARTS_TAR

{'tcode_num': 0,
 'dow': 1,
 'month': 2,
 'day': 3,
 'dtme': 4,
 'td_sc': 5,
 'log_amount_sc': 6}

In [12]:
FIELD_DIMS_NET

{'tcode_num': 16,
 'dow': 7,
 'month': 12,
 'day': 31,
 'dtme': 31,
 'td_sc': 2,
 'log_amount_sc': 2}

In [13]:
FIELD_STARTS_NET

{'tcode_num': 0,
 'dow': 16,
 'month': 23,
 'day': 35,
 'dtme': 66,
 'td_sc': 97,
 'log_amount_sc': 99}

In [14]:
DATA_KEY_ORDER

['tcode_num', 'dow', 'month', 'day', 'dtme', 'td_sc', 'log_amount_sc']

In [15]:
def count_seqs_in_df(df):                   #14354
    gb_aid = df.groupby("account_id")["account_id"]

    full_seqs_per_acct = gb_aid.count() // max_seq_len

    n_full_seqs = sum(full_seqs_per_acct)
    n_part_seqs = sum(gb_aid.count() - full_seqs_per_acct*max_seq_len >= min_seq_len)
    
    return n_full_seqs + n_part_seqs

In [16]:
n_seqs = count_seqs_in_df(df)
n_steps = max_seq_len
n_feat_inp = sum(FIELD_DIMS_IN.values())
n_feat_tar = sum(FIELD_DIMS_TAR.values())

inp_tensor = np.zeros((n_seqs, n_steps, n_feat_inp))
tar_tensor = np.zeros((n_seqs, n_steps, n_feat_tar))

inp_tensor.shape, tar_tensor.shape

((14354, 80, 26), (14354, 80, 7))

In [17]:
def bulk_encode_time_value(val, max_val):
    x = np.sin(2* np.pi/max_val * val)
    y = np.cos(2*np.pi /max_val * val)
    return np.stack([x,y], axis=1)

In [18]:
#seq is group.iloc[start:start+seq_len], group is df.groupby('account_id')
def seq_to_inp_tensor(seq, inp_tensor, seq_i, seq_len):

    for k in DATA_KEY_ORDER:

        depth = FIELD_DIMS_IN[k]
        st = FIELD_STARTS_IN[k]
        enc_type = INP_ENCODINGS[k]
        
        if enc_type == "oh":
            x = tf.one_hot(seq[k], depth).numpy()
        elif enc_type == "cl":
            max_val = CLOCK_DIMS[k]
            x = bulk_encode_time_value(seq[k], max_val)
        elif enc_type == "raw":
            x = np.expand_dims(seq[k], 1)
        else:
            raise Exception(f"Got invalid enc_type: {enc_type}")
            
            

        inp_tensor[seq_i,:seq_len, st:st+depth] = x

In [19]:
def seq_to_targ_tensor(seq, tar_tensor, seq_i, seq_len):

    for k in DATA_KEY_ORDER:
        
        depth = FIELD_DIMS_TAR[k]
        st = FIELD_STARTS_TAR[k]
        enc_type = TAR_ENCODINGS[k]
        
        if enc_type == "cl-i":
            max_val = CLOCK_DIMS[k]
            x = np.expand_dims(seq[k]%max_val, 1)
        elif enc_type == "raw":
            x = np.expand_dims(seq[k], 1)
        else:
            raise Exception(f"Got invalid enc_type: {enc_type}")
            
        
        tar_tensor[seq_i,:seq_len, st:st+depth] = x

In [20]:
seq_i = 0
rows_per_acct = {}
alert_every = 2000
attribute = "age_sc"


attributes = np.zeros(n_seqs)
start_time = time.time()
for acct_id, group in df.groupby("account_id"):
    rows_per_acct[acct_id] = []
    
    for i in range(len(group) // max_seq_len + 1):

        n_trs = len(group)
        start = i*max_seq_len
        seq_len = min(max_seq_len, n_trs - start)   

        if seq_len >= min_seq_len:
            seq_to_inp_tensor(group.iloc[start:start+seq_len], inp_tensor, seq_i, seq_len)
            seq_to_targ_tensor(group.iloc[start:start+seq_len],tar_tensor, seq_i, seq_len)
#             tar_tensor[seq_i,:seq_len,:] = seq_to_targ_tensor(group.iloc[start:start+seq_len])
            attributes[seq_i] = group["age"].iloc[0]

            rows_per_acct[acct_id].append(seq_i)
            seq_i += 1
            
            if seq_i % alert_every == 0:
                print(f"Finished encoding {seq_i} of {n_seqs} seqs")
inp_tensor = np.concatenate([np.repeat(attributes[:, None, None], n_feat_inp, axis=2), 
                             inp_tensor], 
                             axis=1)
print(f"Took {time.time() - start_time:.2f} secs")

2023-10-25 10:51:42.805963: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-10-25 10:51:42.807168: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-10-25 10:51:42.836128: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-10-25 10:51:42.836371: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.45GiB deviceMemoryBandwidth: 625.94GiB/s
2023-10-25 10:51:42.836402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-10-25 10:5

Finished encoding 2000 of 14354 seqs
Finished encoding 4000 of 14354 seqs
Finished encoding 6000 of 14354 seqs
Finished encoding 8000 of 14354 seqs
Finished encoding 10000 of 14354 seqs
Finished encoding 12000 of 14354 seqs
Finished encoding 14000 of 14354 seqs
Took 90.38 secs


In [21]:
inp_tensor[0][0]    # the first element in the sequence 

array([29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29.,
       29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29., 29.])

In [22]:
tar_tensor.shape

(14354, 80, 7)

In [23]:
attributes[:10], attributes.shape

(array([29., 29., 29., 54., 54., 54., 54., 54., 54., 43.]), (14354,))

In [24]:
rows_per_acct

{1: [0, 1, 2],
 2: [3, 4, 5, 6, 7, 8],
 3: [9, 10],
 4: [11, 12, 13],
 5: [14],
 6: [15, 16, 17],
 7: [18, 19],
 8: [20, 21, 22],
 9: [23, 24, 25, 26, 27],
 10: [28, 29],
 11: [30, 31, 32],
 12: [33, 34],
 13: [35],
 14: [36, 37],
 15: [38, 39, 40, 41, 42],
 16: [43],
 17: [44],
 18: [45, 46, 47, 48],
 19: [49, 50, 51, 52],
 20: [53, 54],
 21: [55],
 22: [56, 57, 58, 59],
 23: [60, 61, 62, 63, 64],
 24: [65, 66],
 25: [67, 68, 69, 70],
 26: [71, 72, 73, 74, 75],
 27: [76, 77, 78, 79, 80],
 29: [81, 82, 83, 84, 85],
 30: [86, 87, 88, 89],
 31: [90, 91, 92, 93, 94],
 32: [95, 96, 97],
 33: [98, 99, 100, 101, 102],
 34: [103, 104, 105, 106, 107],
 35: [108, 109, 110, 111, 112],
 36: [113, 114, 115, 116, 117],
 37: [118, 119],
 38: [120, 121],
 39: [122, 123, 124, 125, 126],
 40: [127, 128, 129, 130, 131],
 41: [132],
 42: [133, 134, 135],
 43: [136, 137, 138, 139],
 44: [140, 141, 142, 143],
 45: [144],
 47: [145, 146, 147, 148, 149, 150, 151],
 48: [152, 153],
 49: [154],
 50: [155, 156,

In [25]:
df

,account_id,amount,tcode,datetime,year,month,dow,day,dtme,td,age,log_amount,log_amount_sc,td_sc,age_sc,tcode_num
0,1,1000.0,CREDIT__CREDIT IN CASH__nan,1995-03-24,1995,3,4,24,7,0.0,29,3.000434,2.823750,0.000000,1.745524,0
1,1,3679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-04-13,1995,4,3,13,17,20.0,29,3.565848,3.355869,3.298201,1.745524,1
2,1,12600.0,CREDIT__CREDIT IN CASH__nan,1995-04-23,1995,4,6,23,7,10.0,29,4.100405,3.858949,1.649100,1.745524,0
3,1,19.2,CREDIT__nan__INTEREST CREDITED,1995-04-30,1995,4,6,30,0,7.0,29,1.305351,1.228484,1.154370,1.745524,2
4,1,3679.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-05-13,1995,5,5,13,18,13.0,29,3.565848,3.355869,2.143831,1.745524,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056315,11382,25600.0,DEBIT__CASH WITHDRAWAL__nan,1998-12-02,1998,12,2,2,29,2.0,46,4.408257,4.148672,0.329820,2.768763,3
1056316,11382,46248.0,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1998-12-10,1998,12,3,10,21,8.0,46,4.665102,4.390393,1.319280,2.768763,1
1056317,11382,6300.0,DEBIT__CASH WITHDRAWAL__nan,1998-12-25,1998,12,4,25,6,15.0,46,3.799409,3.575677,2.473651,2.768763,3
1056318,11382,311.3,CREDIT__nan__INTEREST CREDITED,1998-12-31,1998,12,3,31,0,6.0,46,2.494572,2.347677,0.989460,2.768763,2


# Split and create tf dataset

In [25]:
from sklearn.model_selection import train_test_split

x_tr, x_cv, inds_tr, inds_cv, targ_tr, targ_cv = train_test_split(inp_tensor, np.arange(n_seqs), tar_tensor, test_size=0.2)

In [26]:
x_tr.shape, x_cv.shape, inds_cv

((11483, 81, 26),
 (2871, 81, 26),
 array([  218, 13377,  3013, ...,  5220, 11025,  8408]))

In [27]:
ds_tr = tf.data.Dataset.from_tensor_slices((x_tr.astype(np.float32), targ_tr.astype(np.float32)))
ds_cv = tf.data.Dataset.from_tensor_slices((x_cv.astype(np.float32), targ_cv.astype(np.float32)))


In [28]:
ds_tr

<TensorSliceDataset shapes: ((81, 26), (80, 7)), types: (tf.float32, tf.float32)>

In [29]:
def make_batches(ds, buffer_size, batch_size):
    return ds.cache().shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
BUFFER_SIZE = ds_tr.cardinality().numpy()
bs = 64    #batch size
train_batches = make_batches(ds_tr, BUFFER_SIZE, bs)
train_batches

<PrefetchDataset shapes: ((None, 81, 26), (None, 80, 7)), types: (tf.float32, tf.float32)>

## Transformer

In [30]:
#features of class Transformer
ACTIVATIONS = {
    "td_sc": "relu",
    "log_amount_sc": "relu"
}
config = {}
config["ORDER"] = DATA_KEY_ORDER
config["FIELD_STARTS_IN"] = FIELD_STARTS_IN
config["FIELD_DIMS_IN"] = FIELD_DIMS_IN
config["FIELD_STARTS_NET"] = FIELD_STARTS_NET
config["FIELD_DIMS_NET"] = FIELD_DIMS_NET

config["ACTIVATIONS"] = ACTIVATIONS

num_layers_enc = None  # no encoder
num_layers_dec = 4
d_model = 128
num_heads = 2
dff = d_model
maximum_position_encoding = 256
net_info = None
inp_dim = n_feat_inp
final_dim = None
rate=0.1
optimizer = tf.keras.optimizers.Adam()
EARLY_STOP = 2
EPOCHS = 80

In [31]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_loss.reset_states()

In [32]:
# create only one sample of inp and tar to trace one iteration of loop
j = 0
for (batch_no, (inp, tar)) in enumerate(train_batches):
    if j == 0:
       break

batch_no

0

In [33]:
tar

<tf.Tensor: shape=(64, 80, 7), dtype=float32, numpy=
array([[[ 4.        ,  5.        ,  8.        , ...,  0.        ,
          3.463111  ,  1.1228663 ],
        [ 2.        ,  5.        ,  8.        , ...,  0.        ,
          0.        ,  1.7245997 ],
        [ 5.        ,  4.        ,  9.        , ..., 24.        ,
          0.98946023,  3.1327682 ],
        ...,
        [ 3.        ,  3.        ,  0.        , ..., 20.        ,
          0.16491003,  2.6722724 ],
        [ 3.        ,  3.        ,  0.        , ..., 13.        ,
          1.1543703 ,  2.972663  ],
        [ 3.        ,  5.        ,  0.        , ..., 11.        ,
          0.32982007,  2.8772542 ]],

       [[ 3.        ,  4.        ,  7.        , ...,  6.        ,
          2.8034706 ,  3.9813588 ],
        [ 4.        ,  3.        ,  7.        , ...,  0.        ,
          0.98946023,  1.1228663 ],
        [ 2.        ,  3.        ,  7.        , ...,  0.        ,
          0.        ,  2.1574826 ],
        ...,
 

In [34]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len), a triangle matrix, upper are ones and, lower and main diagonal are zeroes

def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(tf.reduce_sum(seq, axis=2), 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_masks(tar):

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return combined_mask, dec_target_padding_mask    # combined_mask : (batch_size, 1, seq_len, seq_len)



In [35]:
# combined_mask : (batch_size, 1, seq_len, seq_len)
# dec_padding_mask : (batch_size, 1, 1, seq_len)
combined_mask, dec_padding_mask = create_masks(tar)   #tar (batch_size, seq_len, 7)

In [ ]:
combined_mask

In [36]:
def get_angles(pos, i, d_model):
    #pos (256,1), 256=maximum_position_encoding
    # i (1, 128), 128=d_model(embedding dimension)
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))    #angle_rates(1,128)
    return pos * angle_rates     #(256,128)
    
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],    #(256,1)
                          np.arange(d_model)[np.newaxis, :],       #(1, 128)
                          d_model)
    #angle_rads (256, 128)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]    #(1,256,128)

    return tf.cast(pos_encoding, dtype=tf.float32)

In [37]:
# positional_encoding(maximum_position_encoding = 256, d_model = 128)
pos_encoding = positional_encoding(maximum_position_encoding, d_model)   #(1,256,128)
pos_encoding[:, :max_seq_len, :]

<tf.Tensor: shape=(1, 80, 128), dtype=float32, numpy=
array([[[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
          1.0000000e+00,  0.0000000e+00,  1.0000000e+00],
        [ 8.4147096e-01,  5.4030228e-01,  7.6172042e-01, ...,
          1.0000000e+00,  1.1547820e-04,  1.0000000e+00],
        [ 9.0929741e-01, -4.1614684e-01,  9.8704624e-01, ...,
          9.9999994e-01,  2.3095640e-04,  1.0000000e+00],
        ...,
        [ 9.9952018e-01, -3.0975033e-02, -6.4864838e-01, ...,
          9.9994731e-01,  8.8917045e-03,  9.9996048e-01],
        [ 5.1397848e-01, -8.5780311e-01, -9.9999952e-01, ...,
          9.9994588e-01,  9.0071773e-03,  9.9995941e-01],
        [-4.4411266e-01, -8.9597094e-01, -6.4716274e-01, ...,
          9.9994451e-01,  9.1226511e-03,  9.9995840e-01]]], dtype=float32)>

In [38]:
inp[:, :-1]

<tf.Tensor: shape=(64, 80, 26), dtype=float32, numpy=
array([[[25.        , 25.        , 25.        , ..., 25.        ,
         25.        , 25.        ],
        [ 0.        ,  0.        ,  0.        , ...,  1.        ,
          3.463111  ,  1.1228663 ],
        [ 0.        ,  0.        ,  1.        , ...,  1.        ,
          0.        ,  1.7245997 ],
        ...,
        [ 0.        ,  1.        ,  0.        , ..., -0.44039416,
          0.65964013,  3.29849   ],
        [ 0.        ,  0.        ,  0.        , ..., -0.61210597,
          0.16491003,  2.6722724 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.8743466 ,
          1.1543703 ,  2.972663  ]],

       [[48.        , 48.        , 48.        , ..., 48.        ,
         48.        , 48.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.34730524,
          2.8034706 ,  3.9813588 ],
        [ 0.        ,  0.        ,  0.        , ...,  1.        ,
          0.98946023,  1.1228663 ],
        ...,


In [39]:
tar_inp = inp[:, :-1] # predict next from this   (64, 80, 26)   , inp:(64, 81, 26)
tar_out = inp[:, 1:]  #(64, 80, 26)
tar_out

<tf.Tensor: shape=(64, 80, 26), dtype=float32, numpy=
array([[[ 0.        ,  0.        ,  0.        , ...,  1.        ,
          3.463111  ,  1.1228663 ],
        [ 0.        ,  0.        ,  1.        , ...,  1.        ,
          0.        ,  1.7245997 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.15142778,
          0.98946023,  3.1327682 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.61210597,
          0.16491003,  2.6722724 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.8743466 ,
          1.1543703 ,  2.972663  ],
        [ 0.        ,  0.        ,  0.        , ..., -0.61210597,
          0.32982007,  2.8772542 ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.34730524,
          2.8034706 ,  3.9813588 ],
        [ 0.        ,  0.        ,  0.        , ...,  1.        ,
          0.98946023,  1.1228663 ],
        [ 0.        ,  0.        ,  1.        , ...,  1.        ,
          0.        ,  2.1574826 ],
        ...,


In [39]:
bs

64

In [40]:
max_seq_len

80

In [40]:
# self.decoder = Decoder(num_layers_dec, d_model, num_heads, dff, maximum_position_encoding, inp_dim,rate)
#dec_output, attention_weights = self.decoder(tar_inp, training, look_ahead_mask, dec_padding_mask)
#seq_len = 80
#input layer
inp = tf.keras.layers.Input((None, inp_dim))           #(batch_size, seq_len, inp_feature)
x = tf.keras.layers.Dense(dff, activation='relu')(inp)       #(batch_size, seq_len, dff=128)
x = tf.keras.layers.Dense(d_model)(x)                        # (batch_size, seq_len, d_model=128)
#seq_len = tf.shape(x)[1]
#seq_len = 80
attention_weights_dic = {}
seq_len = tf.shape(x)[1]
x += pos_encoding[:, :seq_len, :]     #x is the output of Input layer
x = tf.keras.layers.Dropout(rate)(x, training=True)
#Decoder Layer
#Decoder Block1
## MultiHeadAttention
assert d_model % num_heads == 0
depth = d_model // num_heads
q = tf.keras.layers.Dense(d_model)(x)    #Wq
k = tf.keras.layers.Dense(d_model)(x)    #Wk
v = tf.keras.layers.Dense(d_model)(x)    #Wv

#batch_size = bs
batch_size = tf.shape(q)[0]
q = tf.reshape(q, (batch_size, -1, num_heads, depth))
q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

k = tf.reshape(k, (batch_size, -1, num_heads, depth))
k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

v = tf.reshape(v, (batch_size, -1, num_heads, depth))
v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
# scale scores(matmul_qk)
dk = tf.cast(tf.shape(k)[-1], tf.float32)
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

# add the mask to the scaled tensor
scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

# softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

scaled_attention = tf.matmul(attention_weights, v)


scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
output = tf.keras.layers.Dense(d_model)(concat_attention)


attn_weights_block1 = attention_weights
attn1 = output

attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

#point-wise feed forward network
ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

i = 0
attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block1


#Decoder Block2
x = out3
q = tf.keras.layers.Dense(d_model)(x)    #Wq
k = tf.keras.layers.Dense(d_model)(x)    #Wk
v = tf.keras.layers.Dense(d_model)(x)    #Wv

#batch_size = bs
batch_size = tf.shape(q)[0]
q = tf.reshape(q, (batch_size, -1, num_heads, depth))
q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

k = tf.reshape(k, (batch_size, -1, num_heads, depth))
k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

v = tf.reshape(v, (batch_size, -1, num_heads, depth))
v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
# scale scores(matmul_qk)
dk = tf.cast(tf.shape(k)[-1], tf.float32)
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

# add the mask to the scaled tensor
scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

# softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

scaled_attention = tf.matmul(attention_weights, v)


scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
output = tf.keras.layers.Dense(d_model)(concat_attention)


attn_weights_block2 = attention_weights
attn1 = output

attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

#point-wise feed forward network
ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

i = 1
attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block2

#Decoder Block3
x = out3
q = tf.keras.layers.Dense(d_model)(x)    #Wq
k = tf.keras.layers.Dense(d_model)(x)    #Wk
v = tf.keras.layers.Dense(d_model)(x)    #Wv

#batch_size = bs
batch_size = tf.shape(q)[0]
q = tf.reshape(q, (batch_size, -1, num_heads, depth))
q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

k = tf.reshape(k, (batch_size, -1, num_heads, depth))
k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

v = tf.reshape(v, (batch_size, -1, num_heads, depth))
v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
# scale scores(matmul_qk)
dk = tf.cast(tf.shape(k)[-1], tf.float32)
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

# add the mask to the scaled tensor
scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

# softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

scaled_attention = tf.matmul(attention_weights, v)


scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
output = tf.keras.layers.Dense(d_model)(concat_attention)


attn_weights_block3 = attention_weights
attn1 = output

attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

#point-wise feed forward network
ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

i = 2
attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block3

#Decoder Block4
x = out3
q = tf.keras.layers.Dense(d_model)(x)    #Wq
k = tf.keras.layers.Dense(d_model)(x)    #Wk
v = tf.keras.layers.Dense(d_model)(x)    #Wv

#batch_size = bs
batch_size = tf.shape(q)[0]
q = tf.reshape(q, (batch_size, -1, num_heads, depth))
q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

k = tf.reshape(k, (batch_size, -1, num_heads, depth))
k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

v = tf.reshape(v, (batch_size, -1, num_heads, depth))
v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
# scale scores(matmul_qk)
dk = tf.cast(tf.shape(k)[-1], tf.float32)
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

# add the mask to the scaled tensor
scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

# softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

scaled_attention = tf.matmul(attention_weights, v)


scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
output = tf.keras.layers.Dense(d_model)(concat_attention)


attn_weights_block4 = attention_weights
attn1 = output

attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

#point-wise feed forward network
ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

i = 3
attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block4

dec_output = out3

final_output = tf.keras.layers.Dense(d_model, activation=None)(dec_output)
model_pred = tf.keras.models.Model(inp , final_output)


In [42]:
ACTIVATIONS

{'td_sc': 'relu', 'log_amount_sc': 'relu'}

In [43]:
DATA_KEY_ORDER, FIELD_DIMS_IN

(['tcode_num', 'dow', 'month', 'day', 'dtme', 'td_sc', 'log_amount_sc'],
 {'tcode_num': 16,
  'dow': 2,
  'month': 2,
  'day': 2,
  'dtme': 2,
  'td_sc': 1,
  'log_amount_sc': 1})

In [41]:
layers_dict = {}
preds = {}

for name, dim in FIELD_DIMS_NET.items():
    acti = ACTIVATIONS.get(name, None)
    layers_dict[name] = tf.keras.layers.Dense(dim, activation=acti)
final_output2 = final_output
for net_name in DATA_KEY_ORDER:
    #print("Running net", net_name)
    pred = layers_dict[net_name](final_output2)
    #print("pred shape", pred.shape)
    preds[net_name] = pred
    st = FIELD_STARTS_IN[net_name]
    end = st + FIELD_DIMS_IN[net_name]
    to_add = tar_out[:, :, st: end]
    #print("Start and end", st, end)
            
    final_output2 = tf.concat([final_output2, to_add], axis=-1)
    #print("Final output shape after",net_name, "is", final_output.shape, "\n")

In [42]:
preds

{'tcode_num': <KerasTensor: shape=(None, None, 16) dtype=float32 (created by layer 'dense_27')>,
 'dow': <KerasTensor: shape=(64, 80, 7) dtype=float32 (created by layer 'dense_28')>,
 'month': <KerasTensor: shape=(64, 80, 12) dtype=float32 (created by layer 'dense_29')>,
 'day': <KerasTensor: shape=(64, 80, 31) dtype=float32 (created by layer 'dense_30')>,
 'dtme': <KerasTensor: shape=(64, 80, 31) dtype=float32 (created by layer 'dense_31')>,
 'td_sc': <KerasTensor: shape=(64, 80, 2) dtype=float32 (created by layer 'dense_32')>,
 'log_amount_sc': <KerasTensor: shape=(64, 80, 2) dtype=float32 (created by layer 'dense_33')>}

In [43]:
model = tf.keras.models.Model(inputs=inp, outputs=preds)   #this model is used for training

In [44]:
predictions = model(tar_inp)

2023-10-25 10:55:41.633314: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [45]:
model_pred(tar_inp)

<tf.Tensor: shape=(64, 80, 128), dtype=float32, numpy=
array([[[ 2.47508794e-01,  2.31427848e-01,  6.85405135e-01, ...,
         -9.50253308e-01,  1.94203764e-01,  3.38773757e-01],
        [-5.24850368e-01, -3.96497637e-01, -4.07264791e-02, ...,
         -1.21726763e+00,  1.14652133e+00, -5.22654772e-01],
        [-1.32784799e-01, -1.06285310e+00,  8.40427697e-01, ...,
         -2.10446879e-01,  5.04676163e-01,  1.98306844e-01],
        ...,
        [-1.26743758e+00, -1.70405477e-01,  8.92285645e-01, ...,
         -1.52311623e+00,  1.05818474e+00,  5.15485816e-02],
        [-8.73098612e-01, -9.36779007e-03,  5.91618657e-01, ...,
         -9.17498767e-01,  1.25917017e+00, -3.97295803e-01],
        [-1.08385921e+00, -1.88244104e-01, -4.61101979e-01, ...,
         -1.43684518e+00,  1.10385203e+00, -1.38263345e-01]],

       [[-1.36280549e+00, -7.83154786e-01,  8.76789689e-01, ...,
         -9.98257637e-01,  3.76738042e-01,  4.07920986e-01],
        [-6.01806879e-01, -2.85904169e-01,  1.17

In [46]:
predictions

{'tcode_num': <tf.Tensor: shape=(64, 80, 16), dtype=float32, numpy=
 array([[[ 0.74753004,  0.12910928, -0.07689029, ...,  1.1557971 ,
          -1.9513344 ,  1.469992  ],
         [-0.04918024,  0.08208848,  0.88004893, ...,  0.01808494,
          -2.1121235 ,  1.8796155 ],
         [ 1.2310207 , -0.2557822 ,  0.82883936, ...,  0.25696054,
          -1.6881715 ,  1.3324795 ],
         ...,
         [ 1.382555  ,  0.07207727,  0.6905937 , ..., -0.28981024,
          -1.2691842 ,  1.5594574 ],
         [ 2.6756737 , -0.0909652 ,  1.2991941 , ..., -0.469168  ,
          -1.2052423 ,  1.3935015 ],
         [ 2.2972739 , -0.6957778 , -0.478017  , ...,  1.8051989 ,
          -2.094645  ,  2.6022766 ]],
 
        [[ 1.2446796 ,  1.2328284 ,  1.0294368 , ..., -0.84119624,
          -0.22446063,  2.0291536 ],
         [ 1.2034487 , -1.1051345 ,  1.0070878 , ...,  0.6485173 ,
          -1.2314957 ,  2.0270073 ],
         [ 0.6963331 ,  0.29812345,  0.04276058, ...,  0.6608512 ,
          -1.694

In [47]:
FIELD_STARTS_TAR, FIELD_DIMS_TAR

({'tcode_num': 0,
  'dow': 1,
  'month': 2,
  'day': 3,
  'dtme': 4,
  'td_sc': 5,
  'log_amount_sc': 6},
 {'tcode_num': 1,
  'dow': 1,
  'month': 1,
  'day': 1,
  'dtme': 1,
  'td_sc': 1,
  'log_amount_sc': 1})

In [48]:
FIELD_STARTS_IN

{'tcode_num': 0,
 'dow': 16,
 'month': 18,
 'day': 20,
 'dtme': 22,
 'td_sc': 24,
 'log_amount_sc': 25}

In [49]:
FIELD_DIMS_IN

{'tcode_num': 16,
 'dow': 2,
 'month': 2,
 'day': 2,
 'dtme': 2,
 'td_sc': 1,
 'log_amount_sc': 1}

In [50]:
LOSS_TYPES

{'day': 'scce',
 'dtme': 'scce',
 'dow': 'scce',
 'month': 'scce',
 'tcode_num': 'scce',
 'td_sc': 'pdf',
 'log_amount_sc': 'pdf'}

In [51]:
#loss function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return  -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi)
real = tar
loss_parts = []
loss_parts_weighted = []
LOSS_WEIGHTS = {
 'td_sc':1.,
 'month': 0.015,
 'day': 0.025,
 'dtme': 0.025,
 'dow': 0.01,
 'tcode_num': 1.,
 'log_amount_sc': 2.}
for k, k_pred in predictions.items():

    st = FIELD_STARTS_TAR[k]
    end = st + FIELD_DIMS_TAR[k]
    loss_type = LOSS_TYPES[k]
    

    if loss_type == "scce":
        loss_ = SparseCategoricalCrossentropy(from_logits=True, reduction='none')(real[:, :, st:end], k_pred)   #(batch_size, seq_len)
        
    elif loss_type == "pdf":
        loss_ = -log_normal_pdf(real[:, :, st:end], k_pred[:,:,0:1], k_pred[:,:,1:2])[:,:,0]                     #(batch_size, seq_len)
    mask = tf.math.logical_not(tf.math.equal(tf.reduce_sum(real, axis=2), 0))
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    loss_ = tf.reduce_sum(loss_)/tf.reduce_sum(mask)
    loss_parts.append(loss_)
    loss_parts_weighted.append(loss_ * LOSS_WEIGHTS[k])
loss = tf.reduce_sum(loss_parts_weighted)
    
        

In [56]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=13.687483>

In [53]:
# generate data
n_seqs_to_generate= 50
start_date_opts = data.groupby("account_id")["datetime"].min().dt.date.to_list()
start_dates = np.random.choice(start_date_opts, size=n_seqs_to_generate) # sample start dates from real data
seq_ages = np.random.choice(attributes, size=n_seqs_to_generate) # sample ages from real data

In [ ]:
#full_df, seqs, raw = generate_seqs(length= seq_len, ages=seq_ages, start_dates= start_dates, return_single_df=True, greedy_dates=True)

In [54]:
layers_dict = {}
preds = {}

for name, dim in FIELD_DIMS_NET.items():
    acti = ACTIVATIONS.get(name, None)
    layers_dict[name] = tf.keras.layers.Dense(dim, activation=acti)
final_output2 = final_output
for net_name in DATA_KEY_ORDER:
    #print("Running net", net_name)
    pred = layers_dict[net_name](final_output2)
    #print("pred shape", pred.shape)
    preds[net_name] = pred
    st = FIELD_STARTS_IN[net_name]
    end = st + FIELD_DIMS_IN[net_name]
    to_add = tar_out[:, :, st: end]
    #print("Start and end", st, end)
            
    final_output2 = tf.concat([final_output2, to_add], axis=-1)
    #print("Final output shape after",net_name, "is", final_output.shape, "\n")

In [ ]:
n_seqs_to_generate= 50
start_date_opts = data.groupby("account_id")["datetime"].min().dt.date.to_list()
start_dates = np.random.choice(start_date_opts, size=n_seqs_to_generate) # sample start dates from real data
seq_ages = np.random.choice(attributes, size=n_seqs_to_generate) # sample ages from real data
#full_df, seqs, raw = generate_seqs(length= seq_len, ages=seq_ages, start_dates= start_dates, return_single_df=True, greedy_dates=True)
# the body of function generate_seqs()
length = max_seq_len
ages = seq_ages
start_dates = start_dates
greedy_dates = True
return_single_df = True
return_df_list=False
if return_single_df and return_df_list:
    raise Exception("At most one of: 'return_single_df' and 'return_df_list' can be true")
START_DATE = df["datetime"].min()
START_DATE = START_DATE.date()
date_inds = np.array([(d - START_DATE).days for d in start_dates])
max_length = length
outp = np.repeat(np.array(ages)[:, None, None], repeats=n_feat_inp, axis=2) / ATTR_SCALE    #(n_seqs_to_generate, 1, n_feat_inp)
raw_preds = []
raw_preds.append(outp)

date_info = None

i = 0 #(for i in range(max_length))
combined_mask, dec_padding_mask = create_masks(outp) 
#call_to_generate(transformer, outp, True, combined_mask, dec_padding_mask, date_inds, date_info, greedy_dates =greedy_dates)

transformer, attention_weights = createmodel(inp_dim, dff, d_model, rate, num_heads, combined_mask)
final_output = transformer(outp)

### Predict each field  ###
preds_dict = {}
raw_preds_dict = {}
encoded_preds_d = {}
encoded_preds = []

layers_dict = {}
for name, dim in FIELD_DIMS_NET.items():
    acti = ACTIVATIONS.get(name, None)
    layers_dict[name] = tf.keras.layers.Dense(dim, activation=acti) 
for net_name in DATA_KEY_ORDER:
    #print("Running net", net_name)
    pred = layers_dict[net_name](final_output)
    #print("pred shape", pred.shape)
    raw_preds_dict[net_name] = pred
    pred = reencode_net_prediction(net_name, pred) # keeps time step
    preds_dict[net_name] = pred
    encoded_preds_d[net_name] = pred[:,-1,:] 
    encoded_preds.append(pred[:,-1,:])  
    final_output = tf.concat([final_output, pred], axis=2)


encoded_preds_d
      
        


In [105]:
#combined_date_info, inds = raw_dates_to_reencoded(raw_preds_dict, date_inds)
import datetime
import calendar
get_dtme = lambda d: calendar.monthrange(d.year, d.month)[1] - d.day
MAX_YEARS_SPAN = 15
END_DATE = START_DATE.replace(year = START_DATE.year+ MAX_YEARS_SPAN)
ALL_DATES = [START_DATE + datetime.timedelta(i) for i in range((END_DATE - START_DATE).days)]
AD = np.array([(d.month % 12, d.day % 31, d.weekday() % 7, i, d.year, get_dtme(d)) for i, d in enumerate(ALL_DATES)])
max_days = 100
all_ps = [tf.nn.softmax(raw_preds_dict[k][:,-1]).numpy() for k in ["month", "day", "dow", "dtme"]]   #(n_seqs_to_generate, datedim)
timesteps = np.zeros(len(date_inds)).astype(int)
sc = TD_SCALE
for i, (month_ps, day_ps, dow_ps, dtme_ps, td_pred, si) in enumerate(zip(*all_ps, raw_preds_dict["td_sc"][:,-1].numpy(), date_inds)):
        ps = month_ps[AD[si:si+max_days,0]]*day_ps[AD[si:si+max_days,1]]*dow_ps[AD[si:si+max_days,2]] *dtme_ps[AD[si:si+max_days,-1]] * \
                np.exp(log_normal_pdf_gen(AD[si:si+max_days,3]-si, mean = td_pred[0]*sc, logvar=td_pred[1]*sc))
        if greedy_dates:
            timesteps[i] = np.argmax(ps)
        else:
            timesteps[i] = np.random.choice(max_days, p=ps/sum(ps))
inds = date_inds + timesteps

combined_date_info = {}
combined_date_info["td_sc"] = tf.expand_dims(timesteps.astype(np.float32)/ TD_SCALE, axis=1)
combined_date_info["month"] = bulk_encode_time_value(AD[inds, 0], 12)
combined_date_info["day"] = bulk_encode_time_value(AD[inds, 1], 31)
combined_date_info["dow"] = bulk_encode_time_value(AD[inds, 2], 7)
combined_date_info["dtme"] = bulk_encode_time_value(AD[inds, -1], 31)

encoded_preds_d.update(combined_date_info)
l = [encoded_preds_d[k] for k in DATA_KEY_ORDER]
encoded_preds =  tf.expand_dims(tf.concat(l, axis=1), axis=1)    #(n_seqs_to_generate, 1, 26)
enc_preds = tf.reshape(tf.constant(encoded_preds), shape=(-1,1, n_feat_inp))
outp =  tf.concat([outp, enc_preds], axis=1)
outp

        


<tf.Tensor: shape=(50, 3, 26), dtype=float32, numpy=
array([[[ 4.273525  ,  4.273525  ,  4.273525  , ...,  4.273525  ,
          4.273525  ,  4.273525  ],
        [ 0.        ,  1.        ,  0.        , ...,  0.9795299 ,
          2.6385605 ,  0.5842118 ],
        [ 0.        ,  1.        ,  0.        , ...,  0.9795299 ,
          2.6385605 ,  0.5842118 ]],

       [[ 2.6483817 ,  2.6483817 ,  2.6483817 , ...,  2.6483817 ,
          2.6483817 ,  2.6483817 ],
        [ 0.        ,  1.        ,  0.        , ...,  0.34730524,
          1.6491003 ,  0.08843018],
        [ 0.        ,  1.        ,  0.        , ...,  0.34730524,
          1.6491003 ,  0.08843018]],

       [[ 2.4076197 ,  2.4076197 ,  2.4076197 , ...,  2.4076197 ,
          2.4076197 ,  2.4076197 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.15142778,
          1.8140104 ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.15142778,
          1.8140104 ,  0.        ]],

       ...,

       [[ 

In [103]:
enc_preds

<tf.Tensor: shape=(50, 1, 26), dtype=float32, numpy=
array([[[ 0.        ,  1.        ,  0.        , ...,  0.9795299 ,
          2.6385605 ,  0.5842118 ]],

       [[ 0.        ,  1.        ,  0.        , ...,  0.34730524,
          1.6491003 ,  0.08843018]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.15142778,
          1.8140104 ,  0.        ]],

       ...,

       [[ 0.        ,  0.        ,  0.        , ...,  0.68896693,
          2.3087404 ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ..., -0.44039416,
          6.7613115 ,  0.28275937]],

       [[ 1.        ,  0.        ,  0.        , ..., -0.61210597,
          3.6280208 ,  0.        ]]], dtype=float32)>

In [101]:
encoded_preds

<tf.Tensor: shape=(50, 1, 26), dtype=float32, numpy=
array([[[ 0.        ,  1.        ,  0.        , ...,  0.9795299 ,
          2.6385605 ,  0.5842118 ]],

       [[ 0.        ,  1.        ,  0.        , ...,  0.34730524,
          1.6491003 ,  0.08843018]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.15142778,
          1.8140104 ,  0.        ]],

       ...,

       [[ 0.        ,  0.        ,  0.        , ...,  0.68896693,
          2.3087404 ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ..., -0.44039416,
          6.7613115 ,  0.28275937]],

       [[ 1.        ,  0.        ,  0.        , ..., -0.61210597,
          3.6280208 ,  0.        ]]], dtype=float32)>

In [97]:
encoded_preds_d.update(combined_date_info)

In [84]:
def log_normal_pdf_gen(sample, mean, logvar, raxis=1):
    log2pi = tf.cast(tf.math.log(2. * np.pi), tf.float64)
    return  -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi)

In [58]:
# Used when generating data
# Takes the output from BF and transforms it into the 
# correct format to be used as input to BF
def reencode_net_prediction(net_name, predictions):
    
    date_info = {'month':12, 'day':31, 'dtme':31, 'dow':7}
    batch_size = predictions.shape[0]                      ##(n_seqs_to_generate, 1, 1)
    
    
    if net_name in ['td_sc', "log_amount_sc"]:
        return predictions[:, :, 0:1]
    


    elif net_name in date_info.keys():
        dim = FIELD_DIMS_NET[net_name]
        choices = np.arange(dim)
        ps = tf.nn.softmax(predictions, axis=2).numpy().reshape(-1, dim)    #(n_seqs_to_generate, dim of date info)
        
        
        choosen =  np.array([np.random.choice(choices, p=p) for p in ps])   #(n_seqs_to_generate,)
        
        x = bulk_encode_time_value(choosen, max_val=dim)            #(n_seqs_to_generate, 2)
        
        return np.reshape(x, newshape=(batch_size, -1, 2))          #(n_seqs_to_generate, 1, 2)

    
    elif "_num" in net_name:
        dim = FIELD_DIMS_NET[net_name]
        choices = np.arange(dim)
        ps = tf.nn.softmax(predictions, axis=2).numpy().reshape(-1, dim)  #(n_seqs_to_generate, 16)

        
        choosen =  np.reshape([np.random.choice(choices, p=p) for p in ps], newshape=(batch_size, -1))   #(n_seqs_to_generate, 1)
        
        return tf.one_hot(choosen, depth=dim)   #(n_seqs_to_generate, 1, 16)
    
    else:
        raise Exception(f"Got invalid net_name: {net_name}")


In [56]:
def createmodel(inp_dim, dff, d_model, rate, num_heads, combined_mask):
    #input layer
    inp = tf.keras.layers.Input((None, inp_dim))           #(batch_size, seq_len, inp_feature)
    x = tf.keras.layers.Dense(dff, activation='relu')(inp)       #(batch_size, seq_len, dff=128)
    x = tf.keras.layers.Dense(d_model)(x)                        # (batch_size, seq_len, d_model=128)
    #seq_len = tf.shape(x)[1]
    #seq_len = 80
    attention_weights_dic = {}
    seq_len = tf.shape(x)[1]
    x += pos_encoding[:, :seq_len, :]     #x is the output of Input layer
    x = tf.keras.layers.Dropout(rate)(x, training=True)
    #Decoder Layer
    #Decoder Block1
    ## MultiHeadAttention
    assert d_model % num_heads == 0
    depth = d_model // num_heads
    q = tf.keras.layers.Dense(d_model)(x)    #Wq
    k = tf.keras.layers.Dense(d_model)(x)    #Wk
    v = tf.keras.layers.Dense(d_model)(x)    #Wv

    #batch_size = bs
    batch_size = tf.shape(q)[0]
    q = tf.reshape(q, (batch_size, -1, num_heads, depth))
    q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

    k = tf.reshape(k, (batch_size, -1, num_heads, depth))
    k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

    v = tf.reshape(v, (batch_size, -1, num_heads, depth))
    v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

    matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
    # scale scores(matmul_qk)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

    # add the mask to the scaled tensor
    scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

    scaled_attention = tf.matmul(attention_weights, v)


    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
    output = tf.keras.layers.Dense(d_model)(concat_attention)


    attn_weights_block1 = attention_weights
    attn1 = output

    attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

    #point-wise feed forward network
    ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
    ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
    ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

    out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

    i = 0
    attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block1


    #Decoder Block2
    x = out3
    q = tf.keras.layers.Dense(d_model)(x)    #Wq
    k = tf.keras.layers.Dense(d_model)(x)    #Wk
    v = tf.keras.layers.Dense(d_model)(x)    #Wv

    #batch_size = bs
    batch_size = tf.shape(q)[0]
    q = tf.reshape(q, (batch_size, -1, num_heads, depth))
    q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

    k = tf.reshape(k, (batch_size, -1, num_heads, depth))
    k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

    v = tf.reshape(v, (batch_size, -1, num_heads, depth))
    v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

    matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
    # scale scores(matmul_qk)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

    # add the mask to the scaled tensor
    scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

    scaled_attention = tf.matmul(attention_weights, v)


    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
    output = tf.keras.layers.Dense(d_model)(concat_attention)


    attn_weights_block2 = attention_weights
    attn1 = output

    attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

    #point-wise feed forward network
    ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
    ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
    ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

    out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

    i = 1
    attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block2

    #Decoder Block3
    x = out3
    q = tf.keras.layers.Dense(d_model)(x)    #Wq
    k = tf.keras.layers.Dense(d_model)(x)    #Wk
    v = tf.keras.layers.Dense(d_model)(x)    #Wv

    #batch_size = bs
    batch_size = tf.shape(q)[0]
    q = tf.reshape(q, (batch_size, -1, num_heads, depth))
    q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

    k = tf.reshape(k, (batch_size, -1, num_heads, depth))
    k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

    v = tf.reshape(v, (batch_size, -1, num_heads, depth))
    v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

    matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
    # scale scores(matmul_qk)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

    # add the mask to the scaled tensor
    scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

    scaled_attention = tf.matmul(attention_weights, v)


    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
    output = tf.keras.layers.Dense(d_model)(concat_attention)


    attn_weights_block3 = attention_weights
    attn1 = output

    attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

    #point-wise feed forward network
    ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
    ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
    ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

    out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

    i = 2
    attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block3

    #Decoder Block4
    x = out3
    q = tf.keras.layers.Dense(d_model)(x)    #Wq
    k = tf.keras.layers.Dense(d_model)(x)    #Wk
    v = tf.keras.layers.Dense(d_model)(x)    #Wv

    #batch_size = bs
    batch_size = tf.shape(q)[0]
    q = tf.reshape(q, (batch_size, -1, num_heads, depth))
    q = tf.transpose(q, perm=[0,2,1,3])                    #(batch_size, num_heads, seq_len, depth)

    k = tf.reshape(k, (batch_size, -1, num_heads, depth))
    k = tf.transpose(k, perm=[0,2,1,3])                     #(batch_size, num_heads, seq_len, depth)

    v = tf.reshape(v, (batch_size, -1, num_heads, depth))
    v = tf.transpose(v, perm=[0,2,1,3])                      #(batch_size, num_heads, seq_len, depth)

    matmul_qk = tf.matmul(q, k, transpose_b=True)     #scores  (batch_size, num_heads, seq_len_q, seq_len_k)
    # scale scores(matmul_qk)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)     #(batch_size, num_heads, seq_len_q, seq_len_k)

    # add the mask to the scaled tensor
    scaled_attention_logits += (combined_mask * -1e9)           #(batch_size, num_heads, seq_len_q, seq_len_k)

    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)    #(batch_size, num_heads, seq_len_q, seq_len_k)

    scaled_attention = tf.matmul(attention_weights, v)


    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, d_model))  # (batch_size, seq_len_q, d_model)
    output = tf.keras.layers.Dense(d_model)(concat_attention)


    attn_weights_block4 = attention_weights
    attn1 = output

    attn1 = tf.keras.layers.Dropout(rate)(attn1, training=True)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn1 + x)    #Add & Norm

    #point-wise feed forward network
    ffn_output = tf.keras.layers.Dense(dff, activation='relu')(out1)  # (batch_size, seq_len, dff)
    ffn_output = tf.keras.layers.Dense(d_model)(ffn_output)  # (batch_size, seq_len, d_model)
    ffn_output = tf.keras.layers.Dropout(rate)(ffn_output, training=True)

    out3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(ffn_output+out1)   # (batch_size, seq_len, d_model)

    i = 3
    attention_weights_dic['decoder_layer{}_block1'.format(i+1)] = attn_weights_block4

    dec_output = out3

    final_output = tf.keras.layers.Dense(d_model, activation=None)(dec_output)
    model_pred = tf.keras.models.Model(inp , final_output)
    return model_pred, attention_weights_dic
